In [22]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, r'C:\code\astrocam')
import pandas as pd
from pathlib import Path
from fwhm.star_finder import StarFinder
from fwhm.star_matcher import StarMatcher
import matplotlib.pyplot as plt
import re
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
pi_data_file = r"D:\Astro\Objects\C30-NGC7331\fwhm.csv"
df = pd.read_csv(pi_data_file)

In [24]:
fit_list = {x.name:str(x) for x in Path(r"D:\Astro").glob("**/*.fit")}
print(f"Found {len(fit_list)}")

Found 2789


In [25]:
df['fit_file'] = df.File.apply(lambda x: fit_list[re.findall(r'(Light_.*?C).*',x)[0]+".fit"])

In [26]:
df

,Index,Approved,Locked,File,Weight,PSF Signal Weight,PSF Power Weight,SNR Weight,FWHM,Eccentricity,...,Median,Median Mean Deviation,Noise,Noise Ratio,Stars,Star Residual,FWHM Mean Deviation,Eccentricity Mean Deviation,Star Residual Mean Deviation,fit_file
0,14,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_00948_...,0.0,776.0,2810000.0,1.80,3.0627,0.4692,...,16.1,2.88,0.429,0.715,458,0.0193,1.1625,0.1532,0.0104,D:\Astro\20220922\C30\Light\Light_00948_180.0s...
1,27,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_00961_...,0.0,726.0,2490000.0,1.78,3.1120,0.4887,...,16.6,2.96,0.435,0.720,428,0.0198,1.0250,0.1450,0.0109,D:\Astro\20220922\C30\Light\Light_00961_180.0s...
2,21,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_00955_...,0.0,741.0,2610000.0,1.80,3.1677,0.4720,...,16.3,2.91,0.431,0.718,453,0.0208,1.0185,0.1411,0.0103,D:\Astro\20220922\C30\Light\Light_00955_180.0s...
3,18,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_00952_...,0.0,759.0,2700000.0,1.81,3.1840,0.4619,...,16.2,2.90,0.430,0.717,447,0.0211,1.0659,0.1480,0.0103,D:\Astro\20220922\C30\Light\Light_00952_180.0s...
4,19,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_00953_...,0.0,722.0,2460000.0,1.85,3.1876,0.5180,...,16.3,2.90,0.431,0.718,383,0.0226,1.0306,0.1198,0.0103,D:\Astro\20220922\C30\Light\Light_00953_180.0s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,54,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_01310_...,0.0,680.0,2240000.0,1.88,3.9763,0.4168,...,17.1,3.08,0.437,0.726,361,0.0228,1.0214,0.1422,0.0112,D:\Astro\20220927\C30\Light\Light_01310_180.0s...
70,57,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_01317_...,0.0,657.0,2070000.0,1.88,3.9848,0.4075,...,17.5,3.16,0.443,0.731,351,0.0258,0.9513,0.1372,0.0122,D:\Astro\20220927\C30\Light\Light_01317_180.0s...
71,52,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_01308_...,0.0,689.0,2300000.0,1.89,3.9939,0.4116,...,17.0,3.06,0.435,0.727,366,0.0225,1.0908,0.1505,0.0109,D:\Astro\20220927\C30\Light\Light_01308_180.0s...
72,56,True,False,D:/Astro/Objects/C30-NGC7331/subs/Light_01312_...,0.0,681.0,2230000.0,1.88,4.0048,0.4207,...,17.2,3.11,0.439,0.728,361,0.0249,0.9815,0.1423,0.0112,D:\Astro\20220927\C30\Light\Light_01312_180.0s...


In [27]:
print(f"Numer of images: {len(df)}")

Numer of images: 74


In [28]:
starfinder = StarFinder()
starmatcher = StarMatcher()
starData = starfinder.getStarData(df.fit_file[0], topk=40)
star_img = starData["star_mask"]
df_ref = starData["stars"]

RuntimeError: The size of tensor a (4144) must match the size of tensor b (200) at non-singleton dimension 3

In [ ]:
plt.imshow(star_img, cmap='gray')

In [ ]:
df_ref

In [ ]:
def ave_fwhm(fname):
  starData = starfinder.getStarData(fname, topk=20)
  df = starData["stars"]
  return pd.Series({"myfwhm_x": df.fwhm_x.mean(), "myfwhm_y":df.fwhm_y.mean(), "myfwhm_ave":((df.fwhm_x + df.fwhm_y)/2).mean()})

In [ ]:
cols = df.fit_file.apply(lambda x: ave_fwhm(x))
cols

In [ ]:
df[['my_fwhm_x', 'my_fwhm_y', 'my_fwhm']] = cols

In [ ]:
df

In [ ]:
import numpy as np
plt.plot(df.FWHM)
plt.plot(df.my_fwhm)
# plt.plot(df.my_fwhm_y-6.5)
# plt.plot(df.my_fwhm-6.5)

z = np.polyfit(df.index, df.my_fwhm, 1)
print(z)
p = np.poly1d(z)
# add trendline to plot
plt.plot(df.index, p(df.index))

z = np.polyfit(df.index, df.FWHM, 1)
print(z)
p = np.poly1d(z)
# add trendline to plot
plt.plot(df.index, p(df.index))

plt.show()

In [ ]:
df_ref

In [ ]:
df_fwhm = None

for i, tgt_file in enumerate(df.fit_file):
    print(f"Processing {tgt_file}...")
    df_tgt = starfinder.getStarData(tgt_file, topk=10)['stars']

    df_matched = starmatcher.matchStars(df_ref, df_tgt)
     
    if df_fwhm is None:
        df_fwhm = df_matched[['index_ref', 'fwhm_x_ref', 'fwhm_y_ref', 'fwhm_x_tgt', 'fwhm_y_tgt']]
    else:
        df_fwhm = pd.merge(left=df_fwhm, right=df_matched[['index_ref', 'fwhm_x_tgt', 'fwhm_y_tgt']], how='left', on='index_ref', suffixes=('', f'_{i:05d}'))
    
    

df_fwhm.to_csv("fwhm.csv")

In [ ]:
df_fwhm.T.to_csv("fwhm_t.csv")